# Creating a Data Pipeline for a Recommender System using Blue Brain Nexus 

In this notebook, you will create a recommendation engine using Blue Brain Nexus. Using some movie rating data,
you will train a collaborative filtering model for movie recommendation using a given matrix factorization class and export the trained model to Elasticsearch. Once exported, 
you can test your recommendations by querying Elasticsearch and displaying the results.

![Movie Recommendation](recommendation.png)

### _Prerequisites_

The notebook assumes you have installed Nexus SDK (https://github.com/BlueBrain/nexus-python-sdk).


## Overview

You will work through the following steps

1. Set up the Nexus environment in Python 
2. Pull the data from Nexus
3. Prepare the data
4. Train the recommendation model
5. Push the model results to Nexus
6. Show recommendations by querying Nexus

## Step 1: Set up Nexus environment

Blue Brain Nexus provides a Python SDK to facilitate the use of Nexus, including functionalities of authentication and data access, etc. The SDK is available at https://github.com/BlueBrain/nexus-python-sdk.

Here, we will first pip install the sdk. Then, we will set up the Nexus environment in this notebook with a provided access token. 

In [ ]:
# !pip install git+https://github.com/BlueBrain/nexus-python-sdk

In [127]:
import nexussdk as nexus

deployment = 'YOUR ENVIRONMENT'
token = 'YOUR ACCESS TOKEN'

In [128]:
nexus.config.set_environment(deployment)
nexus.config.set_token(token)

## Step 2: Pull data from Nexus

Now we will start pulling data that has been ingested into your Nexus project previously. 

For building a classical recommendation system using matrix factorization, we will need a user-by-item matrix where nonzero elements of the matrix are ratings that a user has given an item. To do that, we will 

1.   Query all the rating data from Nexus
2.   Fetch each rating in a JSON format
3.   Add each data into a DataFrame object

## Step 3: Prepare the data

To get the recommendation right, we must construct and transform the data correctly. This is usually a very important step so that you are sure your machine learning algorithm is consuming the correct data in a good way.

In the case of collaborative filtering using matrix factorization, the preparation of the data contains the following steps:

- As in the U-I matrix we will have the user id and the item id as incremental integers, we will assign a unique number between (0, #users) to each user and do the same for movies. The mapping between the id in the U-I matrix will be stored, which can be further used to query the recommendation. 


- Then, we will create the U-I matrix by assigning each user's rating to each movie on a zero matrix created using numpy.


- Finally, we will split the data into training and testing. This is done by removing 10 ratings for each user and assign them to the test set. 


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from numpy.linalg import solve

np.random.seed(0)

In [41]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('../data/ml-latest-small/ratings.csv', names=names)
df = df.drop(0).astype(np.float)

item_mapping = dict( enumerate(df.item_id.astype('category').cat.categories) )
inv_item_mapping = {v: k for k, v in item_mapping.items()}

df.user_id = df.user_id.astype('category').cat.codes.values
df.item_id = df.item_id.astype('category').cat.codes.values

In [83]:
idx_to_movie = {}
movie_df = pd.read_csv('/Users/hulu/Downloads/ml-latest-small/movies.csv')
for k, v in item_mapping.items():
    idx_to_movie[k] = movie_df[movie_df.movieId==v].title.values[0]

In [51]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

# Create r_{ui}, our ratings matrix
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

# Split into training and test sets. 
# Remove 10 ratings for each user 
# and assign them to the test set
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)

## Step 4: Train a recommmender model on the ratings data

Your data is now prepared as a U-I matrix and you will use it to build a collaborative filtering recommendation model.

Collaborative filtering is a recommendation approach that is effectively based on the "wisdom of the crowd". It makes the assumption that, if two people share similar preferences, then the things that one of them prefers could be good recommendations to make to the other. In other words, if user A tends to like certain movies, and user B shares some of these preferences with user A, then the movies that user A likes, that user B has not yet seen, may well be movies that user B will also like.

In a similar manner, we can think about items as being similar if they tend to be rated highly by the same people, on average.

Hence these models are based on the combined, collaborative preferences and behavior of all users in aggregate. They tend to be very effective in practice (provided you have enough preference data to train the model). The ratings data you have is a form of explicit preference data, perfect for training collaborative filtering models. 

Matrix factorization (MF) is a classical method to perform collaborative filtering model. The core idea of MF is to represent the ratings as a user-item ratings matrix. In the diagram below you will see this matrix on the left (with users as rows and movies as columns). The entries in this matrix are the ratings given by users to movies. 

You may also notice that the matrix has missing entries because not all users have rated all movies. In this situation we refer to the data as sparse.

![alt text]()

MF methods aim to find two much smaller matrices (one representing the users and the other the items) that, when multiplied together, re-construct the original ratings matrix as closely as possible. This is know as factorizing the original matrix, hence the name of the technique.

The two smaller matrices are called factor matrices (or latent features). The user and movie factor matrices are illustrated on the right in the diagram above. The idea is that each user factor vector is a compressed representation of the user's preferences and behavior. Likewise, each item factor vector is a compressed representation of the item. Once the model is trained, the factor vectors can be used to make recommendations, which is what you will do in the following sections.

The optimization of the MF can be done using different methods. In this example, we will use 2 popular methods:

- Alternating Least Squares (ALS)

- Stochastic Gradient Descent (SGD)


Further reading:

[Explicit Matrix Factorization: ALS, SGD, and All That Jazz](https://www.ethanrosenthal.com/2016/01/09/explicit-matrix-factorization-sgd-als/)

[ALS Implicit Collaborative Filtering
](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)


Below we have provided an explicit MF class which can perform MF with both ALS and SGD optimizations. 

In [54]:
class ExplicitMF():
    def __init__(self, 
                 ratings,
                 n_factors=40,
                 learning='sgd',
                 item_fact_reg=0.0, 
                 user_fact_reg=0.0,
                 item_bias_reg=0.0,
                 user_bias_reg=0.0,
                 verbose=False):
        """
        Train a matrix factorization model to predict empty 
        entries in a matrix. The terminology assumes a 
        ratings matrix which is ~ user x item
        
        Params
        ======
        ratings : (ndarray)
            User x Item matrix with corresponding ratings
        
        n_factors : (int)
            Number of latent factors to use in matrix 
            factorization model
        learning : (str)
            Method of optimization. Options include 
            'sgd' or 'als'.
        
        item_fact_reg : (float)
            Regularization term for item latent factors
        
        user_fact_reg : (float)
            Regularization term for user latent factors
            
        item_bias_reg : (float)
            Regularization term for item biases
        
        user_bias_reg : (float)
            Regularization term for user biases
        
        verbose : (bool)
            Whether or not to printout training progress
        """
        
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.item_fact_reg = item_fact_reg
        self.user_fact_reg = user_fact_reg
        self.item_bias_reg = item_bias_reg
        self.user_bias_reg = user_bias_reg
        self.learning = learning
        if self.learning == 'sgd':
            self.sample_row, self.sample_col = self.ratings.nonzero()
            self.n_samples = len(self.sample_row)
        self._v = verbose

    def als_step(self,
                 latent_vectors,
                 fixed_vecs,
                 ratings,
                 _lambda,
                 type='user'):
        """
        One of the two ALS steps. Solve for the latent vectors
        specified by type.
        """
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI), 
                                             ratings[u, :].dot(fixed_vecs))
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda
            
            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI), 
                                             ratings[:, i].T.dot(fixed_vecs))
        return latent_vectors

    def train(self, n_iter=10, learning_rate=0.1):
        """ Train model for n_iter iterations from scratch."""
        # initialize latent vectors        
        self.user_vecs = np.random.normal(scale=1./self.n_factors,\
                                          size=(self.n_users, self.n_factors))
        self.item_vecs = np.random.normal(scale=1./self.n_factors,
                                          size=(self.n_items, self.n_factors))
        
        if self.learning == 'als':
            self.partial_train(n_iter)
        elif self.learning == 'sgd':
            self.learning_rate = learning_rate
            self.user_bias = np.zeros(self.n_users)
            self.item_bias = np.zeros(self.n_items)
            self.global_bias = np.mean(self.ratings[np.where(self.ratings != 0)])
            self.partial_train(n_iter)
    
    
    def partial_train(self, n_iter):
        """ 
        Train model for n_iter iterations. Can be 
        called multiple times for further training.
        """
        ctr = 1
        while ctr <= n_iter:
            if ctr % 10 == 0 and self._v:
                print ('\tcurrent iteration: {}'.format(ctr))
            if self.learning == 'als':
                self.user_vecs = self.als_step(self.user_vecs, 
                                               self.item_vecs, 
                                               self.ratings, 
                                               self.user_fact_reg, 
                                               type='user')
                self.item_vecs = self.als_step(self.item_vecs, 
                                               self.user_vecs, 
                                               self.ratings, 
                                               self.item_fact_reg, 
                                               type='item')
            elif self.learning == 'sgd':
                self.training_indices = np.arange(self.n_samples)
                np.random.shuffle(self.training_indices)
                self.sgd()
            ctr += 1

    def sgd(self):
        for idx in self.training_indices:
            u = self.sample_row[idx]
            i = self.sample_col[idx]
            prediction = self.predict(u, i)
            e = (self.ratings[u,i] - prediction) # error
            
            # Update biases
            self.user_bias[u] += self.learning_rate * \
                                (e - self.user_bias_reg * self.user_bias[u])
            self.item_bias[i] += self.learning_rate * \
                                (e - self.item_bias_reg * self.item_bias[i])
            
            #Update latent factors
            self.user_vecs[u, :] += self.learning_rate * \
                                    (e * self.item_vecs[i, :] - \
                                     self.user_fact_reg * self.user_vecs[u,:])
            self.item_vecs[i, :] += self.learning_rate * \
                                    (e * self.user_vecs[u, :] - \
                                     self.item_fact_reg * self.item_vecs[i,:])
    def predict(self, u, i):
        """ Single user and item prediction."""
        if self.learning == 'als':
            return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
        elif self.learning == 'sgd':
            prediction = self.global_bias + self.user_bias[u] + self.item_bias[i]
            prediction += self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
            return prediction
    
    def predict_all(self):
        """ Predict ratings for every user and item."""
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
                
        return predictions
    
    def calculate_learning_curve(self, iter_array, test, learning_rate=0.1):
        """
        Keep track of MSE as a function of training iterations.
        
        Params
        ======
        iter_array : (list)
            List of numbers of iterations to train for each step of 
            the learning curve. e.g. [1, 5, 10, 20]
        test : (2D ndarray)
            Testing dataset (assumed to be user x item).
        
        The function creates two new class attributes:
        
        train_mse : (list)
            Training data MSE values for each value of iter_array
        test_mse : (list)
            Test data MSE values for each value of iter_array
        """
        iter_array.sort()
        self.train_mse =[]
        self.test_mse = []
        iter_diff = 0
        for (i, n_iter) in enumerate(iter_array):
            if self._v:
                print ('Iteration: {}'.format(n_iter))
            if i == 0:
                self.train(n_iter - iter_diff, learning_rate)
            else:
                self.partial_train(n_iter - iter_diff)

            predictions = self.predict_all()

            self.train_mse += [get_mse(predictions, self.ratings)]
            self.test_mse += [get_mse(predictions, test)]
            if self._v:
                print ('Train mse: ' + str(self.train_mse[-1]))
                print ('Test mse: ' + str(self.test_mse[-1]))
            iter_diff = n_iter

In [55]:
best_als_model = ExplicitMF(ratings, n_factors=20, learning='als', \
                            item_fact_reg=0.01, user_fact_reg=0.01)
best_als_model.train(50)

In [56]:
best_sgd_model = ExplicitMF(ratings, n_factors=80, learning='sgd', \
                            item_fact_reg=0.01, user_fact_reg=0.01, \
                            user_bias_reg=0.01, item_bias_reg=0.01)
best_sgd_model.train(200, learning_rate=0.001)

## Step 5: Push the embedding matrix to Nexus

Now that we have the models trained, we can now push the embedding matrices back to Nexus and have them indexed in the  ElasticSearch view. 

## Step 6: Show recommendation by querying Nexus

In [57]:
def cosine_similarity(model):
    sim = model.item_vecs.dot(model.item_vecs.T)
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return sim / norms / norms.T

als_sim = cosine_similarity(best_als_model)
sgd_sim = cosine_similarity(best_sgd_model)

In [111]:
def display_top_k_movies_name(similarity, mapper, movie_idx, k=5):
    print('The recommended films for user who likes "%s"' % (mapper[movie_idx]))
    
    movie_indices = np.argsort(similarity[movie_idx,:])[::-1]
    images = ''
    k_ctr = 0
    # Start i at 1 to not grab the input movie
    i = 1
    while k_ctr < 5:
        movie = mapper[movie_indices[i]]
        print(' - ' + movie)
        k_ctr += 1
        i += 1

In [112]:
movie_id = 5 # Heat
display_top_k_movies_name(als_sim, idx_to_movie, movie_id)

The recommended films for user who likes "Heat (1995)"
 - Up Close and Personal (1996)
 - Last Supper, The (1995)
 - Malice (1993)
 - Speed (1994)
 - Eye for an Eye (1996)


In [117]:
iter_array = [1, 2, 5, 10, 25, 50, 100, 200]
latent_factors = [5, 10, 20, 40, 80]
regularizations = [0.001, 0.01, 0.1, 1.]
regularizations.sort()

best_params = {}
best_params['n_factors'] = latent_factors[0]
best_params['reg'] = regularizations[0]
best_params['n_iter'] = 0
best_params['train_mse'] = np.inf
best_params['test_mse'] = np.inf
best_params['model'] = None

for fact in latent_factors:
    print ('Factors: {}'.format(fact))
    for reg in regularizations:
        print ('Regularization: {}'.format(reg))
        MF_SGD = ExplicitMF(train, n_factors=fact, learning='als',\
                            user_fact_reg=reg, item_fact_reg=reg, \
                            user_bias_reg=reg, item_bias_reg=reg)
        MF_SGD.calculate_learning_curve(iter_array, test, learning_rate=0.001)
        min_idx = np.argmin(MF_SGD.test_mse)
        if MF_SGD.test_mse[min_idx] < best_params['test_mse']:
            best_params['n_factors'] = fact
            best_params['reg'] = reg
            best_params['n_iter'] = iter_array[min_idx]
            best_params['train_mse'] = MF_SGD.train_mse[min_idx]
            best_params['test_mse'] = MF_SGD.test_mse[min_idx]
            best_params['model'] = MF_SGD
            print ('New optimal hyperparameters')
            print (pd.Series(best_params))

Factors: 5
Regularization: 0.001
New optimal hyperparameters
model        <__main__.ExplicitMF object at 0x1a0a196a58>
n_factors                                               5
n_iter                                                  5
reg                                                 0.001
test_mse                                           10.851
train_mse                                         7.37505
dtype: object
Regularization: 0.01
Regularization: 0.1
Regularization: 1.0
Factors: 10
Regularization: 0.001
New optimal hyperparameters
model        <__main__.ExplicitMF object at 0x1a0a1969b0>
n_factors                                              10
n_iter                                                 25
reg                                                 0.001
test_mse                                          10.4631
train_mse                                         6.50026
dtype: object
Regularization: 0.01
Regularization: 0.1
Regularization: 1.0
Factors: 20
Regularization: 0.0